# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-b77b0db429-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Book Workflow Series ("Authors comparison explorative search") 

Consider the following exploratory scenario:


>  Investigate the the production of Paul Auster and Ian McEwan, check how many books they have written for each litarature genre, gather information about their production and about their works which are not books



## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wd:Q47461344`| writtenwork   | node |
| `wd:Q214642`  | Paul Auster   | node |
| `wd:Q190379`  | Ian McEwan    | node |




Also consider

```
wd:Q214642 ?rel ?obj  . 
```
is the BGP to retrieve all **properties of Paul Auster**


The workload should


1. Identify the BGP for obtaining the books (with publishing date and genre) published by the two authors

2. Did the authors published a book in the same year? What is the longest period without publishing a book for the two authors? 

3. Did the authors produced, acted or directed a film? If so, did they write the screenplay?

4. How many films were derived from the books of these two authors? 

5. Which author won more literature-related awards? Have they ever being nominated for a Nobel award? 

In [1]:
# start your workflow here

In [2]:
queryString = """
SELECT COUNT( ?obj)
WHERE { 
wd:Q214642 ?rel ?obj  . 
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '191')]


1

<h3> TASK 1 STARTS HERE </h3>
Identify the BGP for obtaining the books (with publishing date and genre) published by the two authors

Retrieve some Auster's properties.

In [3]:
#Q1
#have a look at some predicates related to Auster
queryString = """
SELECT DISTINCT ?p ?pname WHERE { 

wd:Q214642 ?p ?o  . 

#Labels
?p <http://schema.org/name> ?pname .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('pname', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('pname', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pname', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('pname', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pname', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P1150'), ('pname', 'Regensburg Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('pname', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1233'), ('pname', 'Internet Speculative Fiction Database author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('pname', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1266'), ('pname', 'AlloCiné person ID')]
[('p', 'http://www.wikidata.org

100

Retrieve some Auster's properties but excluding literals.

In [4]:
#Q2
#have a look at some predicates related to Auster, excluding literals
queryString = """
SELECT DISTINCT ?p ?pname WHERE { 

wd:Q214642 ?p ?o  . 
FILTER(!isLiteral(?o))

#Labels
?p <http://schema.org/name> ?pname .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pname', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pname', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('pname', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pname', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pname', 'place of birth')]
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('pname', 'sex or gender')]
[('p', 'http://www.wikidata.org/prop/direct/P26'), ('pname', 'spouse')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('pname', 'country of citizenship')]
[('p', 'http://www.wikidata

24

Check reverse path (book->author or something like that).

In [5]:
#Q3
#check reverse path (book->author or something like that)
#wd:Q47461344
queryString = """
SELECT DISTINCT ?p ?pname WHERE { 

wd:Q47461344 ?p wd:Q214642. 

#Labels
?p <http://schema.org/name> ?pname .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
Empty


0

Book could be instance of writtenwork, add this to the previous BGP.

In [6]:
#Q4
#no results because probably book is instance of writtenwork, check this
queryString = """
SELECT DISTINCT ?p ?pname WHERE { 

?book wdt:P31 wd:Q47461344. 
?book ?p wd:Q214642. 

#Labels
?p <http://schema.org/name> ?pname .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P50'), ('pname', 'author')]


1

Retrieve how many books are written by Auster.

In [7]:
#Q5
#retrieve how many books are written by Auster
queryString = """
SELECT (COUNT(*) AS ?books) WHERE { 

?book wdt:P31 wd:Q47461344;
      wdt:P50 wd:Q214642. 
} 
"""

print("Results")
run_query(queryString)

Results
[('books', '16')]


1

Retrieve predicates related to books written by Auster.

In [2]:
#Q6
#retrieve predicates related to books written by Auster
queryString = """
SELECT DISTINCT ?p ?pname WHERE { 

?book wdt:P31 wd:Q47461344;
      wdt:P50 wd:Q214642;
      ?p ?o.
      
#Labels
?p <http://schema.org/name> ?pname .

}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1085'), ('pname', 'LibraryThing work ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1104'), ('pname', 'number of pages')]
[('p', 'http://www.wikidata.org/prop/direct/P123'), ('pname', 'publisher')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('pname', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('pname', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pname', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pname', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pname', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1814'), ('pname', 'name in kana')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('pname', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2191'), ('pname', 'Vegetti Catalog of Fantastic Literature NILF ID')]
[('p', 

43

In the previous result set I found Genre (P136) and Pubblication Date (P577).Retrieve title, genre and date of books written by Auster and McEwan. 

In [4]:
#Q7
#i found out Genre (P136) and Pubblication Date (P577)
#retrieve title, genre and date of books written by Auster and McEwan
queryString = """
SELECT ?author ?title ?genre ?gname ?date WHERE { 

?book wdt:P31 wd:Q47461344;
      wdt:P50 ?author;
      wdt:P136 ?genre;
      wdt:P1476 ?title;
      wdt:P577 ?date.
      
VALUES ?author {wd:Q190379 wd:Q214642}

#Labels
?genre <http://schema.org/name> ?gname .

}
ORDER BY ?title ?date
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('author', 'http://www.wikidata.org/entity/Q190379'), ('title', 'Amsterdam'), ('genre', 'http://www.wikidata.org/entity/Q8261'), ('gname', 'novel'), ('date', '1998-12-01T00:00:00Z')]
[('author', 'http://www.wikidata.org/entity/Q190379'), ('title', 'Atonement'), ('genre', 'http://www.wikidata.org/entity/Q8261'), ('gname', 'novel'), ('date', '2001-01-01T00:00:00Z')]
[('author', 'http://www.wikidata.org/entity/Q190379'), ('title', 'Black Dogs'), ('genre', 'http://www.wikidata.org/entity/Q8261'), ('gname', 'novel'), ('date', '1992-07-01T00:00:00Z')]
[('author', 'http://www.wikidata.org/entity/Q190379'), ('title', 'Enduring Love'), ('genre', 'http://www.wikidata.org/entity/Q8261'), ('gname', 'novel'), ('date', '1997-01-01T00:00:00Z')]
[('author', 'http://www.wikidata.org/entity/Q214642'), ('title', 'In the Country of Last Things'), ('genre', 'http://www.wikidata.org/entity/Q465821'), ('gname', 'epistolary novel'), ('date', '1987-01-01T00:00:00Z')]
[('author', 'http://www.wikidata.o

30

As before but using group concat on genre, optional on date and gnere.

In [7]:
#Q8
#as previous but use group concat on genre
#add also optional on date and genre
queryString = """
SELECT ?author ?title ?genres ?date WHERE{
  
?book wdt:P50 ?author;
      wdt:P1476 ?title.
OPTIONAL { ?book wdt:P577 ?date.}

{SELECT ?book (GROUP_CONCAT(?genre;SEPARATOR=",") AS ?genres) WHERE { 

?book wdt:P31 wd:Q47461344;
      wdt:P50 ?author.
  
VALUES ?author {wd:Q190379 wd:Q214642}
  
OPTIONAL { ?book wdt:P136 ?genre.}  
}
GROUP BY (?book)}

}
ORDER BY ?title
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('author', 'http://www.wikidata.org/entity/Q190379'), ('title', 'Amsterdam'), ('genres', 'http://www.wikidata.org/entity/Q8261'), ('date', '1998-12-01T00:00:00Z')]
[('author', 'http://www.wikidata.org/entity/Q190379'), ('title', 'Atonement'), ('genres', 'http://www.wikidata.org/entity/Q8261'), ('date', '2001-01-01T00:00:00Z')]
[('author', 'http://www.wikidata.org/entity/Q190379'), ('title', 'Black Dogs'), ('genres', 'http://www.wikidata.org/entity/Q8261'), ('date', '1992-07-01T00:00:00Z')]
[('author', 'http://www.wikidata.org/entity/Q190379'), ('title', 'Enduring Love'), ('genres', 'http://www.wikidata.org/entity/Q8261'), ('date', '1997-01-01T00:00:00Z')]
[('author', 'http://www.wikidata.org/entity/Q214642'), ('title', 'In the Country of Last Things'), ('genres', 'http://www.wikidata.org/entity/Q465821,http://www.wikidata.org/entity/Q12132683,http://www.wikidata.org/entity/Q26928598'), ('date', '1987-01-01T00:00:00Z')]
[('author', 'http://www.wikidata.org/entity/Q214642'), ('t

24

Retrieve how many books are written by McEwan.

In [8]:
#Q9
#retrieve how many books are written by McEwan
queryString = """
SELECT (COUNT(*) AS ?books) WHERE { 

?book wdt:P31 wd:Q47461344;
      wdt:P50 wd:Q190379. 
} 
"""

print("Results")
run_query(queryString)

Results
[('books', '12')]


1

Result of Q8 should be 28.

In [9]:
#Q10
#total result of Q8 should be 28 
queryString = """
SELECT (COUNT(*) AS ?books) WHERE { 

?book wdt:P31 wd:Q47461344;
      wdt:P1476 ?title;
      wdt:P50 ?author. 
VALUES ?author {wd:Q190379 wd:Q214642}
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('books', '24')]


1

Some books don't have title property so i'll re-run Q8 just using label instead of title.

In [10]:
#Q11
#some books don't have title property so i'll re-run Q8 just using label instead of title
queryString = """
SELECT ?author ?book ?bname ?genres ?date WHERE{
  
?book wdt:P50 ?author.
OPTIONAL { ?book wdt:P577 ?date.}

{SELECT ?book (GROUP_CONCAT(?genre;SEPARATOR=",") AS ?genres) WHERE { 

?book wdt:P31 wd:Q47461344;
      wdt:P50 ?author.
  
VALUES ?author {wd:Q190379 wd:Q214642}
  
OPTIONAL { ?book wdt:P136 ?genre.}  
}
GROUP BY (?book)}

#Labels
?book <http://schema.org/name> ?bname .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('author', 'http://www.wikidata.org/entity/Q214642'), ('book', 'http://www.wikidata.org/entity/Q1951470'), ('bname', 'Mr. Vertigo'), ('genres', 'http://www.wikidata.org/entity/Q8261'), ('date', '1994-04-05T00:00:00Z')]
[('author', 'http://www.wikidata.org/entity/Q214642'), ('book', 'http://www.wikidata.org/entity/Q492991'), ('bname', 'Timbuktu'), ('genres', 'http://www.wikidata.org/entity/Q8261,http://www.wikidata.org/entity/Q149537'), ('date', '1999-05-01T00:00:00Z')]
[('author', 'http://www.wikidata.org/entity/Q214642'), ('book', 'http://www.wikidata.org/entity/Q581485'), ('bname', 'The Book of Illusions'), ('genres', 'http://www.wikidata.org/entity/Q8261'), ('date', '2002-01-01T00:00:00Z')]
[('author', 'http://www.wikidata.org/entity/Q214642'), ('book', 'http://www.wikidata.org/entity/Q526792'), ('bname', 'Travels in the Scriptorium'), ('genres', 'http://www.wikidata.org/entity/Q8261'), ('date', '2006-01-01T00:00:00Z')]
[('author', 'http://www.wikidata.org/entity/Q190379'),

28

<h3> TASK 1 ENDS HERE </h3>
In Q11 I retrieved book name, genres and pubblication date. To do that, i used P136 and P577, i also found out P1476 (title) but some books don't have
it so i prefered to use the label.

<h3> TASK 2 STARTS HERE </h3>
Did the authors published a book in the same year? What is the longest period without publishing a book for the two authors? 

Check if some books have the same pubblication date.

In [14]:
#Q13
#check if some books have the same pubblication date
queryString = """
SELECT DISTINCT ?book ?date1 WHERE {

?book wdt:P31 wd:Q47461344;
      wdt:P50 ?a1;
      wdt:P577 ?date1.
VALUES ?a1 {wd:Q190379 wd:Q214642}

{SELECT ?b2 ?date2 WHERE { 

?b2 wdt:P31 wd:Q47461344;
      wdt:P50 ?a2;
      wdt:P577 ?date2.      
VALUES ?a2 {wd:Q190379 wd:Q214642}}}

FILTER( (?date1=?date2) && (?a1!=?a2) )
}
ORDER BY ?date1
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
Empty


0

Pubblication date is a single day so it's logical to have no results, try to change previous query to just consider the year.

In [16]:
#Q14
#pubblication date is a single day so it's logical to have no results, try to change previous query to just consider the year
queryString = """
SELECT ?b1 ?b2 ?y1 ?y2
WHERE  { 
{SELECT ?b1 ?y1 WHERE{
?b1 wdt:P31 wd:Q47461344;
      wdt:P50 wd:Q190379;
      wdt:P577 ?date1.
BIND(YEAR(?date1) AS ?y1)
}}
{SELECT ?b2 ?y2 WHERE{ 
?b2 wdt:P31 wd:Q47461344;
      wdt:P50 wd:Q214642;
      wdt:P577 ?date2.
BIND(YEAR(?date2) AS ?y2)} 
}
FILTER((?b1!=?b2)&&(?y1=?y2))
}
"""

print("Results")
run_query(queryString)

Results
[('b1', 'http://www.wikidata.org/entity/Q3819411'), ('b2', 'http://www.wikidata.org/entity/Q1951470'), ('y1', '1994'), ('y2', '1994')]
[('b1', 'http://www.wikidata.org/entity/Q1747269'), ('b2', 'http://www.wikidata.org/entity/Q1506280'), ('y1', '1987'), ('y2', '1987')]
[('b1', 'http://www.wikidata.org/entity/Q1747269'), ('b2', 'http://www.wikidata.org/entity/Q1215203'), ('y1', '1987'), ('y2', '1987')]
[('b1', 'http://www.wikidata.org/entity/Q1723053'), ('b2', 'http://www.wikidata.org/entity/Q1211751'), ('y1', '2005'), ('y2', '2005')]
[('b1', 'http://www.wikidata.org/entity/Q1704438'), ('b2', 'http://www.wikidata.org/entity/Q1821899'), ('y1', '1992'), ('y2', '1992')]
[('b1', 'http://www.wikidata.org/entity/Q1531861'), ('b2', 'http://www.wikidata.org/entity/Q1322083'), ('y1', '2010'), ('y2', '2010')]


6

Add a constraint to check that also the author is different.

In [18]:
#Q15
#add a constraint to check that also the author is different
queryString = """
SELECT ?b1 ?b2 ?y1
WHERE  { 
{SELECT ?b1 ?y1 ?a1 WHERE{
?b1 wdt:P31 wd:Q47461344;
      wdt:P50 wd:Q190379;
      wdt:P50 ?a1;
      wdt:P577 ?date1.
BIND(YEAR(?date1) AS ?y1)
}}
{SELECT ?b2 ?y2 ?a2 WHERE{ 
?b2 wdt:P31 wd:Q47461344;
      wdt:P50 wd:Q214642;
      wdt:P50 ?a2;
      wdt:P577 ?date2.
BIND(YEAR(?date2) AS ?y2)
} 
}
FILTER((?a1!=?a2)&&(?y1=?y2))
}
ORDER BY ?y1
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('b1', 'http://www.wikidata.org/entity/Q1747269'), ('b2', 'http://www.wikidata.org/entity/Q1506280'), ('y1', '1987')]
[('b1', 'http://www.wikidata.org/entity/Q1747269'), ('b2', 'http://www.wikidata.org/entity/Q1215203'), ('y1', '1987')]
[('b1', 'http://www.wikidata.org/entity/Q1704438'), ('b2', 'http://www.wikidata.org/entity/Q1821899'), ('y1', '1992')]
[('b1', 'http://www.wikidata.org/entity/Q3819411'), ('b2', 'http://www.wikidata.org/entity/Q1951470'), ('y1', '1994')]
[('b1', 'http://www.wikidata.org/entity/Q1723053'), ('b2', 'http://www.wikidata.org/entity/Q1211751'), ('y1', '2005')]
[('b1', 'http://www.wikidata.org/entity/Q1531861'), ('b2', 'http://www.wikidata.org/entity/Q1322083'), ('y1', '2010')]


6

Retrieve the chronological order of books written by Auster.

In [19]:
#Q16
#chronological order of books written by Auster
queryString = """
SELECT DISTINCT ?book ?year WHERE { 

?book wdt:P31 wd:Q47461344;
      wdt:P50 wd:Q214642;
      wdt:P577 ?date.
BIND(YEAR(?date) AS ?year)

}
ORDER BY ?year
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('book', 'http://www.wikidata.org/entity/Q1215203'), ('year', '1987')]
[('book', 'http://www.wikidata.org/entity/Q1506280'), ('year', '1987')]
[('book', 'http://www.wikidata.org/entity/Q1944147'), ('year', '1989')]
[('book', 'http://www.wikidata.org/entity/Q1214916'), ('year', '1990')]
[('book', 'http://www.wikidata.org/entity/Q1821899'), ('year', '1992')]
[('book', 'http://www.wikidata.org/entity/Q1951470'), ('year', '1994')]
[('book', 'http://www.wikidata.org/entity/Q492991'), ('year', '1999')]
[('book', 'http://www.wikidata.org/entity/Q581485'), ('year', '2002')]
[('book', 'http://www.wikidata.org/entity/Q1258502'), ('year', '2003')]
[('book', 'http://www.wikidata.org/entity/Q1211751'), ('year', '2005')]
[('book', 'http://www.wikidata.org/entity/Q526792'), ('year', '2006')]
[('book', 'http://www.wikidata.org/entity/Q1817920'), ('year', '2008')]
[('book', 'http://www.wikidata.org/entity/Q1584652'), ('year', '2009')]
[('book', 'http://www.wikidata.org/entity/Q1322083'), ('yea

14

Retrieve the chronological order of books written by McEwan.

In [22]:
#Q17
#chronological order of books written by McEwan
queryString = """
SELECT DISTINCT ?book ?year WHERE { 

?book wdt:P31 wd:Q47461344;
      wdt:P50 wd:Q190379;
      wdt:P577 ?date.
BIND(YEAR(?date) AS ?year)

}
ORDER BY ?year
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('book', 'http://www.wikidata.org/entity/Q1198186'), ('year', '1978')]
[('book', 'http://www.wikidata.org/entity/Q1754953'), ('year', '1981')]
[('book', 'http://www.wikidata.org/entity/Q1747269'), ('year', '1987')]
[('book', 'http://www.wikidata.org/entity/Q1704438'), ('year', '1992')]
[('book', 'http://www.wikidata.org/entity/Q3819411'), ('year', '1994')]
[('book', 'http://www.wikidata.org/entity/Q2920448'), ('year', '1997')]
[('book', 'http://www.wikidata.org/entity/Q478433'), ('year', '1998')]
[('book', 'http://www.wikidata.org/entity/Q306619'), ('year', '2001')]
[('book', 'http://www.wikidata.org/entity/Q1723053'), ('year', '2005')]
[('book', 'http://www.wikidata.org/entity/Q451491'), ('year', '2007')]
[('book', 'http://www.wikidata.org/entity/Q1531861'), ('year', '2010')]
[('book', 'http://www.wikidata.org/entity/Q972609'), ('year', '2012')]


12

Retrieve the longest time interval between the publication of two books written by Auster.

In [3]:
#Q18
#retrieve the longest time interval between the publication of two books written by Auster
queryString = """
SELECT ((?y1-?y2) AS ?dif) ?y1 ?y2 WHERE { 

#Select and compare  the year difference between two books  
{SELECT ?b1 ?y1 WHERE{
?b1 wdt:P31 wd:Q47461344;
      wdt:P50 wd:Q214642;
      wdt:P577 ?d1.
BIND(YEAR(?d1) AS ?y1)
}}
{SELECT ?b2 ?y2 WHERE{
?b2 wdt:P31 wd:Q47461344;
      wdt:P50 wd:Q214642;
      wdt:P577 ?d2.
BIND(YEAR(?d2) AS ?y2)
}}
#Check that there isn't any other book published in between  
FILTER NOT EXISTS
{
?b3 wdt:P31 wd:Q47461344;
    wdt:P50 wd:Q214642;
    wdt:P577 ?d3.
BIND(YEAR(?d3) AS ?y3)
FILTER( (?y1>?y3) && (?y2<?y3) )
}
}
ORDER BY DESC(?dif)
LIMIT 1
"""

print("Results")
run_query(queryString)

Results
The operation failed EndPointInternalError: endpoint returned code 500 and response. 

Response:
b"Virtuoso 42000 Error The estimated execution time 4970529 (sec) exceeds the limit of 800 (sec).\n\nSPARQL query:\n\n##-b77b0db429-##\nPREFIX wd: <http://www.wikidata.org/entity/> \nPREFIX wdt: <http://www.wikidata.org/prop/direct/> \nPREFIX sc: <http://schema.org/>\n\n\nSELECT ((?y1-?y2) AS ?dif) ?y1 ?y2 WHERE { \n\n#Select and compare  the year difference between two books  \n{SELECT ?b1 ?y1 WHERE{\n?b1 wdt:P31 wd:Q47461344;\n      wdt:P50 wd:Q214642;\n      wdt:P577 ?d1.\nBIND(YEAR(?d1) AS ?y1)\n}}\n{SELECT ?b2 ?y2 WHERE{\n?b2 wdt:P31 wd:Q47461344;\n      wdt:P50 wd:Q214642;\n      wdt:P577 ?d2.\nBIND(YEAR(?d2) AS ?y2)\n}}\n#Check that there isn't any other book published in between  \nFILTER NOT EXISTS\n{\n?b3 wdt:P31 wd:Q47461344;\n    wdt:P50 wd:Q214642;\n    wdt:P577 ?d3.\nBIND(YEAR(?d3) AS ?y3)\nFILTER( (?y1>?y3) && (?y2<?y3) )\n}\n}\nORDER BY DESC(?dif)\nLIMIT 1\n"


<h3> TASK 2 ENDS HERE </h3>
They published six books in the same year; the longest time interval between one book and another one is 5 years for Auster (94-99) 
and 6 for McEwan (81-87). On a side note Q18 is syntactically valid but it runs out of time, so I computed by hand the time interval.

<h3> TASK 3 STARTS HERE </h3>
Did the authors produced, acted or directed a film? If so, did they write the screenplay?

In [2]:
#Q19
#have a look at some predicates
queryString = """
SELECT DISTINCT ?p ?pname WHERE { 

?s ?p ?author.
VALUES ?author {wd:Q214642 wd:Q190379}

#Labels
?p <http://schema.org/name> ?pname .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P162'), ('pname', 'producer')]
[('p', 'http://www.wikidata.org/prop/direct/P170'), ('pname', 'creator')]
[('p', 'http://www.wikidata.org/prop/direct/P22'), ('pname', 'father')]
[('p', 'http://www.wikidata.org/prop/direct/P26'), ('pname', 'spouse')]
[('p', 'http://www.wikidata.org/prop/direct/P2789'), ('pname', 'connects with')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('pname', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P50'), ('pname', 'author')]
[('p', 'http://www.wikidata.org/prop/direct/P57'), ('pname', 'director')]
[('p', 'http://www.wikidata.org/prop/direct/P58'), ('pname', 'screenwriter')]
[('p', 'http://www.wikidata.org/prop/direct/P737'), ('pname', 'influenced by')]
[('p', 'http://www.wikidata.org/prop/direct/P921'), ('pname', 'main subjec

16

Retrieve the films where one of authors acted in using P161 found above.

In [4]:
#Q20
#retrieve the films where one of authors acted in (P161 found above)
queryString = """
SELECT ?author ?film ?fname WHERE { 

?film wdt:P161 ?author.
VALUES ?author {wd:Q214642 wd:Q190379}

#Labels
?film <http://schema.org/name> ?fname .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('author', 'http://www.wikidata.org/entity/Q214642'), ('film', 'http://www.wikidata.org/entity/Q2915460'), ('fname', 'Act of God')]
[('author', 'http://www.wikidata.org/entity/Q214642'), ('film', 'http://www.wikidata.org/entity/Q2364684'), ('fname', 'The Music of Chance')]
[('author', 'http://www.wikidata.org/entity/Q190379'), ('film', 'http://www.wikidata.org/entity/Q7348342'), ('fname', 'The Unbelievers')]


3

Music of chance is also one of the book written by Auster, i want to check if there is a property which connects book and film.

In [5]:
#Q21
#Music of chance is also one of the book written by Auster, i want to check if there is a property which connects book and film
queryString = """
SELECT ?p ?o ?oname WHERE { 

wd:Q2364684 ?p ?o.

#Labels
?o <http://schema.org/name> ?oname .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('o', 'http://www.wikidata.org/entity/Q214642'), ('oname', 'Paul Auster')]
[('p', 'http://www.wikidata.org/prop/direct/P58'), ('o', 'http://www.wikidata.org/entity/Q214642'), ('oname', 'Paul Auster')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('o', 'http://www.wikidata.org/entity/Q329784'), ('oname', 'Charles Durning')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('o', 'http://www.wikidata.org/entity/Q267097'), ('oname', 'Mandy Patinkin')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('o', 'http://www.wikidata.org/entity/Q498420'), ('oname', 'M. Emmet Walsh')]
[('p', 'http://www.wikidata.org/prop/direct/P162'), ('o', 'http://www.wikidata.org/entity/Q7149902'), ('oname', 'Paul Colichman')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('o', 'http://www.wikidata.org/entity/Q11424'), ('oname', 'film')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('o', 'http://www.wikidata.org/entity/Q130232')

20

Retrieve the films where one of authors produced it (P162 found above).

In [6]:
#Q22
#retrieve the films where one of authors produced it (P162 found above)
queryString = """
SELECT ?author ?film ?fname WHERE { 

?film wdt:P162 ?author.
VALUES ?author {wd:Q214642 wd:Q190379}

#Labels
?film <http://schema.org/name> ?fname .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('author', 'http://www.wikidata.org/entity/Q214642'), ('film', 'http://www.wikidata.org/entity/Q3213673'), ('fname', 'The Inner Life of Martin Frost')]
[('author', 'http://www.wikidata.org/entity/Q190379'), ('film', 'http://www.wikidata.org/entity/Q1626186'), ('fname', 'Atonement')]


2

Also Atonement is a book, re-try Q21 with it.

In [7]:
#Q23
#also Atonement is a book, re-try Q21 with it
queryString = """
SELECT ?p ?o ?oname WHERE { 

wd:Q1626186 ?p ?o.

#Labels
?o <http://schema.org/name> ?oname .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('o', 'http://www.wikidata.org/entity/Q193659'), ('oname', 'James McAvoy')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('o', 'http://www.wikidata.org/entity/Q247915'), ('oname', 'Patrick Kennedy')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('o', 'http://www.wikidata.org/entity/Q5980945'), ('oname', 'Ian Bonar')]
[('p', 'http://www.wikidata.org/prop/direct/P750'), ('o', 'http://www.wikidata.org/entity/Q1329615'), ('oname', 'UIP-Dunafilm')]
[('p', 'http://www.wikidata.org/prop/direct/P57'), ('o', 'http://www.wikidata.org/entity/Q118817'), ('oname', 'Joe Wright')]
[('p', 'http://www.wikidata.org/prop/direct/P162'), ('o', 'http://www.wikidata.org/entity/Q1351291'), ('oname', 'Eric Fellner')]
[('p', 'http://www.wikidata.org/prop/direct/P344'), ('o', 'http://www.wikidata.org/entity/Q1359957'), ('oname', 'Seamus McGarvey')]
[('p', 'http://www.wikidata.org/prop/direct/P162'), ('o', 'http://www.wikidata.org/entity/Q1

85

So the P136 + Q52207399 is the pattern for retrieving based on a novel, while P144 has a book as object. Retrieve the films where one of authors was the director (P57 found in Q19).

In [8]:
#Q24
#P136 + Q52207399 (film based on a novel), P144 links to the book
#retrieve the films where one of authors was the director (P57 found in Q19)
queryString = """
SELECT ?author ?film ?fname WHERE { 

?film wdt:P57 ?author.
VALUES ?author {wd:Q214642 wd:Q190379}

#Labels
?film <http://schema.org/name> ?fname .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('author', 'http://www.wikidata.org/entity/Q214642'), ('film', 'http://www.wikidata.org/entity/Q2478977'), ('fname', 'Blue in the Face')]
[('author', 'http://www.wikidata.org/entity/Q214642'), ('film', 'http://www.wikidata.org/entity/Q3213673'), ('fname', 'The Inner Life of Martin Frost')]
[('author', 'http://www.wikidata.org/entity/Q214642'), ('film', 'http://www.wikidata.org/entity/Q653447'), ('fname', 'Smoke')]
[('author', 'http://www.wikidata.org/entity/Q214642'), ('film', 'http://www.wikidata.org/entity/Q1877215'), ('fname', 'Lulu on the Bridge')]


4

Retrieve the films where one of authors was the screenwriter (P58 found in Q19).

In [15]:
#Q25
#retrieve the films where one of authors was the screenwriter (P58 found in Q19)
queryString = """
SELECT ?author ?film ?fname WHERE { 

?film wdt:P58 ?author.
VALUES ?author {wd:Q214642 wd:Q190379}

#Labels
?film <http://schema.org/name> ?fname .
}
ORDER BY ?author
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('author', 'http://www.wikidata.org/entity/Q190379'), ('film', 'http://www.wikidata.org/entity/Q589713'), ('fname', 'The Good Son')]
[('author', 'http://www.wikidata.org/entity/Q190379'), ('film', 'http://www.wikidata.org/entity/Q129813'), ('fname', 'Enduring Love')]
[('author', 'http://www.wikidata.org/entity/Q190379'), ('film', 'http://www.wikidata.org/entity/Q7757528'), ('fname', "The Ploughman's Lunch")]
[('author', 'http://www.wikidata.org/entity/Q190379'), ('film', 'http://www.wikidata.org/entity/Q1879989'), ('fname', 'The Comfort of Strangers')]
[('author', 'http://www.wikidata.org/entity/Q190379'), ('film', 'http://www.wikidata.org/entity/Q27959336'), ('fname', 'On Chesil Beach')]
[('author', 'http://www.wikidata.org/entity/Q190379'), ('film', 'http://www.wikidata.org/entity/Q1626186'), ('fname', 'Atonement')]
[('author', 'http://www.wikidata.org/entity/Q190379'), ('film', 'http://www.wikidata.org/entity/Q1916423'), ('fname', 'The Cement Garden')]
[('author', 'http://w

16

Retrieve the films where one of the authors had a role and also was the screenwriter (P58 found in Q19).

In [10]:
#Q26
#retrieve the films where one of the authors had a role and also was the screenwriter (P58 found in Q19)
queryString = """
SELECT DISTINCT ?author ?film WHERE { 

?film ?p ?author.
VALUES ?p {wdt:P161 wdt:P162 wdt:P57}
VALUES ?author {wd:Q214642 wd:Q190379}

?film wdt:P58 ?author.

}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('author', 'http://www.wikidata.org/entity/Q214642'), ('film', 'http://www.wikidata.org/entity/Q2478977')]
[('author', 'http://www.wikidata.org/entity/Q214642'), ('film', 'http://www.wikidata.org/entity/Q2915460')]
[('author', 'http://www.wikidata.org/entity/Q214642'), ('film', 'http://www.wikidata.org/entity/Q3213673')]
[('author', 'http://www.wikidata.org/entity/Q214642'), ('film', 'http://www.wikidata.org/entity/Q2364684')]
[('author', 'http://www.wikidata.org/entity/Q214642'), ('film', 'http://www.wikidata.org/entity/Q653447')]
[('author', 'http://www.wikidata.org/entity/Q214642'), ('film', 'http://www.wikidata.org/entity/Q1877215')]
[('author', 'http://www.wikidata.org/entity/Q190379'), ('film', 'http://www.wikidata.org/entity/Q1626186')]


7

Retrieve the films where one of the authors had a role and the film was based on one a novel written by him.

In [11]:
#Q27
#retrieve the films where one of the authors had a role and the film was based on one a novel written by him
queryString = """
SELECT DISTINCT ?author ?film ?fname WHERE { 

?film ?p ?author;
      wdt:P144 ?book.
?book wdt:P50 ?author.

VALUES ?p {wdt:P161 wdt:P162 wdt:P57 wdt:P58}
VALUES ?author {wd:Q214642 wd:Q190379}

#Labels
?film <http://schema.org/name> ?fname .

}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('author', 'http://www.wikidata.org/entity/Q190379'), ('film', 'http://www.wikidata.org/entity/Q27959336'), ('fname', 'On Chesil Beach')]
[('author', 'http://www.wikidata.org/entity/Q190379'), ('film', 'http://www.wikidata.org/entity/Q1626186'), ('fname', 'Atonement')]
[('author', 'http://www.wikidata.org/entity/Q190379'), ('film', 'http://www.wikidata.org/entity/Q1916423'), ('fname', 'The Cement Garden')]
[('author', 'http://www.wikidata.org/entity/Q190379'), ('film', 'http://www.wikidata.org/entity/Q28923729'), ('fname', 'The Children Act')]


4

Retrieve how many films are based on a novel written by one of the two authors.

In [13]:
#Q28
#retrieve how many films are based on a novel written by one of the two authors
queryString = """
SELECT ?author (COUNT(?film) AS ?films) WHERE { 

?film wdt:P144 ?book.
?book wdt:P50 ?author.
VALUES ?author {wd:Q214642 wd:Q190379}

}
GROUP BY ?author
ORDER BY DESC(?films)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('author', 'http://www.wikidata.org/entity/Q190379'), ('films', '4')]


1

I want to see what results retrieves the pattern P136 + Q52207399 (genre=based on a novel) found on Q23.

In [14]:
#Q29
#i want to check P136 + Q52207399 (genre=based on a novel) found on Q23
queryString = """
SELECT DISTINCT ?author ?film ?fname WHERE { 

?film ?p ?author;
      wdt:P136 wd:Q52207399.

VALUES ?p {wdt:P161 wdt:P162 wdt:P57 wdt:P58}
VALUES ?author {wd:Q214642 wd:Q190379}

#Labels
?film <http://schema.org/name> ?fname .

}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('author', 'http://www.wikidata.org/entity/Q190379'), ('film', 'http://www.wikidata.org/entity/Q1626186'), ('fname', 'Atonement')]
[('author', 'http://www.wikidata.org/entity/Q190379'), ('film', 'http://www.wikidata.org/entity/Q1916423'), ('fname', 'The Cement Garden')]


2

Count films where one of authors was the screenwriter (P58 found in Q19).

In [16]:
#Q30
#count films where one of authors was the screenwriter (P58 found in Q19)
queryString = """
SELECT ?author (COUNT(?film) AS ?films) WHERE { 

?film wdt:P58 ?author.
FILTER ((?author=wd:Q214642) ||(?author=wd:Q190379))

}
GROUP BY ?author
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('author', 'http://www.wikidata.org/entity/Q214642'), ('films', '7')]
[('author', 'http://www.wikidata.org/entity/Q190379'), ('films', '9')]


2

<h3>TASK 3 AND TASK 4</h3>
I found the following properties: P161 (acted in), P162 (producer), P57 (director) and P58 (screenwriter) and I discovered that 
Auster acted in 2 films and he directed one. He also was the screenwriter in seven productions. I didn't find any film based on his novels but I think this is due to lack of data since he wrote "Music of chance" and also a acted in a film called "Music of chance".
On the other hand McEwan acted in one film, produced one and directed four. He was also the screenwriter in 9 films. On top of that four films are based on his novels and he had a relevant role in all of them.
The property that link a film to the book it is based on is P144.

<h3> TASK 5 STARTS HERE </h3>
Which author won more literature-related awards? Have they ever being nominated for a Nobel award? 

Try to find predicates about nomination/awards.

In [20]:
#Q31
#check predicates about nomination/awards
queryString = """
SELECT DISTINCT ?p ?pname WHERE { 

?author ?p ?o.
VALUES ?author {wd:Q214642 wd:Q190379}
FILTER(!isLiteral(?o))
#Labels
?p <http://schema.org/name> ?pname .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pname', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pname', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P140'), ('pname', 'religion')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pname', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('pname', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')]
[('p', 'http://www.wikidata.org/prop/direct/P172'), ('pname', 'ethnic group')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pname', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pname', 'place of birth')]
[('p', 'http://www.wikidata.org

30

I found P166 (award received) and P1411 (nominated for). Retrieve some award properties.

In [21]:
#Q32
#i found out P166 (award received) and P1411 (nominated for)
queryString = """
SELECT DISTINCT ?p ?pname WHERE { 

?author wdt:P1411 ?award.
?award ?p ?o.
VALUES ?author {wd:Q214642 wd:Q190379}

#Labels
?p <http://schema.org/name> ?pname .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('pname', 'replaced by')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pname', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('pname', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1448'), ('pname', 'official name')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P1705'), ('pname', 'native label')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2121'), ('pname', 'prize money')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('pname', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('pname', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('pname', 'Bibliothèque nationale de Fran

30

Retrieve some properties of award super class.

In [22]:
#Q33
#i found out P166 (award received) and P1411 (nominated for)
queryString = """
SELECT DISTINCT ?p ?pname WHERE { 

?author wdt:P1411 ?award.
?award wdt:P31 ?s.
?s ?p ?o.
VALUES ?author {wd:Q214642 wd:Q190379}

#Labels
?p <http://schema.org/name> ?pname .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pname', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('pname', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pname', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pname', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('pname', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2347'), ('pname', 'YSO ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('pname', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('pname', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('pname', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('p',

34

Retrieve the award won by Auster and its topic.

In [23]:
#Q32
#award won by Auster + topic (P910)
queryString = """
SELECT DISTINCT ?award ?aname ?topic ?tname WHERE { 

wd:Q214642 wdt:P166 ?award.
?award wdt:P910 ?topic.

#Labels
?award <http://schema.org/name> ?aname .
?topic <http://schema.org/name> ?tname .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q1188661'), ('aname', 'PEN/Faulkner Award for Fiction'), ('topic', 'http://www.wikidata.org/entity/Q8708086'), ('tname', 'Category:PEN/Faulkner Foundation awards')]
[('award', 'http://www.wikidata.org/entity/Q52382875'), ('aname', 'AAAS Fellow'), ('topic', 'http://www.wikidata.org/entity/Q6619589'), ('tname', 'Category:Fellows of the American Academy of Arts and Sciences')]
[('award', 'http://www.wikidata.org/entity/Q13452531'), ('aname', 'Commandeur des Arts et des Lettres\u200e'), ('topic', 'http://www.wikidata.org/entity/Q9145202'), ('tname', 'Category:Commandeurs of the Ordre des Arts et des Lettres')]


3

Try to see what results we can obtain using P527.

In [25]:
#Q33
#award categories for recipients P(2517)
queryString = """
SELECT DISTINCT ?award ?aname ?c ?cname WHERE { 

wd:Q214642 wdt:P166 ?award.
?award wdt:P2517 ?c.

#Labels
?award <http://schema.org/name> ?aname .
?c <http://schema.org/name> ?cname .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q3332454'), ('aname', 'Princess of Asturias Literary Prize'), ('c', 'http://www.wikidata.org/entity/Q17337380'), ('cname', 'Category:Recipients of the Prince of Asturias Awards')]
[('award', 'http://www.wikidata.org/entity/Q52382875'), ('aname', 'AAAS Fellow'), ('c', 'http://www.wikidata.org/entity/Q6619589'), ('cname', 'Category:Fellows of the American Academy of Arts and Sciences')]
[('award', 'http://www.wikidata.org/entity/Q13452531'), ('aname', 'Commandeur des Arts et des Lettres\u200e'), ('c', 'http://www.wikidata.org/entity/Q9145202'), ('cname', 'Category:Commandeurs of the Ordre des Arts et des Lettres')]


3

Retrieve some properties which have an award as object.

In [26]:
#Q34
#reverse path
queryString = """
SELECT DISTINCT ?p ?pname WHERE { 

wd:Q214642 wdt:P166 ?award.
?s ?p ?award.

#Labels
?p <http://schema.org/name> ?pname .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pname', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('pname', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('pname', 'member of')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P971'), ('pname', 'category combines topics')]
[('p', 'http://www.wikidata.org/prop/direct/P3730'), ('pname', 'next higher rank')]


8

To distinguish literature award could be an useful to look the connection between book and award instead of author and award.

In [27]:
#Q35
#to disntiguish literature award could be an ideal to look the connection between book and award instead of author and award
queryString = """
SELECT DISTINCT ?award ?aname WHERE { 

?book wdt:P50 wd:Q214642;
      wdt:P166 ?award.

#Labels
?a <http://schema.org/name> ?aname .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
Empty


0

Count nominations for each author.

In [28]:
#Q36
#count nominations for each author
queryString = """
SELECT ?author (COUNT(?n) AS ?nomination) WHERE { 

?author wdt:P1411 ?n.
FILTER ((?author=wd:Q214642) ||(?author=wd:Q190379))
} 
GROUP BY ?author
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('author', 'http://www.wikidata.org/entity/Q190379'), ('nomination', '4')]


1

Count awards won for each author.

In [29]:
#Q37
#count awards won for each author
queryString = """
SELECT ?author (COUNT(?n) AS ?awards) WHERE { 

?author wdt:P166 ?n.
FILTER ((?author=wd:Q214642) ||(?author=wd:Q190379))
} 
GROUP BY ?author
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('author', 'http://www.wikidata.org/entity/Q214642'), ('awards', '5')]
[('author', 'http://www.wikidata.org/entity/Q190379'), ('awards', '15')]


2

Again retrieve some award properties.

In [30]:
#Q38
#have a look at predicates of awards
queryString = """
SELECT DISTINCT ?p ?pname WHERE { 

?author wdt:P166 ?award.
?award ?p ?o.
VALUES ?author {wd:Q214642 wd:Q190379}

#Labels
?p <http://schema.org/name> ?pname .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3729'), ('pname', 'next lower rank')]
[('p', 'http://www.wikidata.org/prop/direct/P112'), ('pname', 'founded by')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pname', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('pname', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1438'), ('pname', 'Jewish Encyclopedia ID (Russian)')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P1705'), ('pname', 'native label')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pname', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1813'), ('pname', 'short name')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2121'), ('pname', 'prize money')]
[('p', 'http://w

41

Try to use some properties found above to distinguish the literary awards.

In [32]:
#Q39
#part of
queryString = """
SELECT DISTINCT ?award ?o ?oname WHERE { 

?author wdt:P166 ?award.
?award wdt:P361 ?o.
VALUES ?author {wd:Q214642 wd:Q190379}

#Labels
?o <http://schema.org/name> ?oname .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q3405076'), ('o', 'http://www.wikidata.org/entity/Q58352'), ('oname', 'Prix Médicis')]
[('award', 'http://www.wikidata.org/entity/Q13452531'), ('o', 'http://www.wikidata.org/entity/Q716909'), ('oname', 'Ordre des Arts et des Lettres')]
[('award', 'http://www.wikidata.org/entity/Q3404580'), ('o', 'http://www.wikidata.org/entity/Q18945'), ('oname', 'Prix Femina')]
[('award', 'http://www.wikidata.org/entity/Q3332454'), ('o', 'http://www.wikidata.org/entity/Q329157'), ('oname', 'Princess of Asturias Awards')]
[('award', 'http://www.wikidata.org/entity/Q12201477'), ('o', 'http://www.wikidata.org/entity/Q14420'), ('oname', 'Order of the British Empire')]


5

In [2]:
#Q40
#P31
queryString = """
SELECT DISTINCT ?award ?aname ?o ?oname WHERE { 

?author wdt:P166 ?award.
?award wdt:P31 ?o.
VALUES ?author {wd:Q214642 wd:Q190379}

#Labels
?award <http://schema.org/name> ?aname .
?o <http://schema.org/name> ?oname .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q15271633'), ('aname', 'Fellow of the Royal Society of Arts'), ('o', 'http://www.wikidata.org/entity/Q618779'), ('oname', 'award')]
[('award', 'http://www.wikidata.org/entity/Q5153503'), ('aname', 'Common Wealth Award of Distinguished Service'), ('o', 'http://www.wikidata.org/entity/Q618779'), ('oname', 'award')]
[('award', 'http://www.wikidata.org/entity/Q50622024'), ('aname', 'honorary doctorate of Carlos III University'), ('o', 'http://www.wikidata.org/entity/Q618779'), ('oname', 'award')]
[('award', 'http://www.wikidata.org/entity/Q56494487'), ('aname', 'honorary doctor of the University of Sussex'), ('o', 'http://www.wikidata.org/entity/Q618779'), ('oname', 'award')]
[('award', 'http://www.wikidata.org/entity/Q891461'), ('aname', 'Bollinger Everyman Wodehouse Prize'), ('o', 'http://www.wikidata.org/entity/Q618779'), ('oname', 'award')]
[('award', 'http://www.wikidata.org/entity/Q277787'), ('aname', 'Jerusalem Prize'), ('o', 'http:

23

wd:Q378427 is for literary award. Count literary awards won for each author.

In [3]:
#Q41
#wd:Q378427 is for literary award
#count literary awards won for each author
queryString = """
SELECT ?author (COUNT(?n) AS ?awards) WHERE { 

?author wdt:P166 ?n.
?n wdt:P31 wd:Q378427.
FILTER ((?author=wd:Q214642) ||(?author=wd:Q190379))
} 
GROUP BY ?author
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('author', 'http://www.wikidata.org/entity/Q214642'), ('awards', '2')]
[('author', 'http://www.wikidata.org/entity/Q190379'), ('awards', '7')]


2

Check reverse path from award to author using P1346 (found on Q38).

In [4]:
#Q42
#check reverse path from award to author using P1346 (found on Q38)
queryString = """
SELECT ?author (COUNT(?award) AS ?awards) WHERE { 

?award wdt:P1346 ?author;
       wdt:P31 wd:Q378427.
       
FILTER ((?author=wd:Q214642) ||(?author=wd:Q190379))
} 
GROUP BY ?author
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('author', 'http://www.wikidata.org/entity/Q190379'), ('awards', '2')]


1

Count nomination for literary awards won for each author.

In [6]:
#Q43
#count nomination for literary awards won for each author
queryString = """
SELECT ?author (COUNT(?n) AS ?nominations) WHERE { 

?author wdt:P1411 ?n.
?n wdt:P31 wd:Q378427.
FILTER ((?author=wd:Q214642) ||(?author=wd:Q190379))
} 
GROUP BY ?author
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('author', 'http://www.wikidata.org/entity/Q190379'), ('nominations', '3')]


1

Retrieve the award name for which McEwan has been nominated.

In [7]:
#Q44
#check nominations for McEwan
queryString = """
SELECT ?n ?name WHERE { 

wd:Q190379 wdt:P1411 ?n.
?n wdt:P31 wd:Q378427.

#Labels
?n <http://schema.org/name> ?name .

} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('n', 'http://www.wikidata.org/entity/Q160082'), ('name', 'Booker Prize')]
[('n', 'http://www.wikidata.org/entity/Q1362723'), ('name', 'International IMPAC Dublin Literary Award')]
[('n', 'http://www.wikidata.org/entity/Q2052291'), ('name', 'International Booker Prize')]


3

Have a look at the awards won by both authors.

In [9]:
#Q45
#have a look at the awards won by both authors
queryString = """
SELECT ?author ?award ?name WHERE { 

?author wdt:P166 ?award.
?award wdt:P31 wd:Q378427.
VALUES ?author {wd:Q214642 wd:Q190379}

#Labels
?award <http://schema.org/name> ?name .

} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('author', 'http://www.wikidata.org/entity/Q214642'), ('award', 'http://www.wikidata.org/entity/Q3332454'), ('name', 'Princess of Asturias Literary Prize')]
[('author', 'http://www.wikidata.org/entity/Q190379'), ('award', 'http://www.wikidata.org/entity/Q39477935'), ('name', 'Fellow of the Royal Society of Literature')]
[('author', 'http://www.wikidata.org/entity/Q214642'), ('award', 'http://www.wikidata.org/entity/Q1188661'), ('name', 'PEN/Faulkner Award for Fiction')]
[('author', 'http://www.wikidata.org/entity/Q190379'), ('award', 'http://www.wikidata.org/entity/Q160082'), ('name', 'Booker Prize')]
[('author', 'http://www.wikidata.org/entity/Q190379'), ('award', 'http://www.wikidata.org/entity/Q392586'), ('name', 'James Tait Black Memorial Prize')]
[('author', 'http://www.wikidata.org/entity/Q190379'), ('award', 'http://www.wikidata.org/entity/Q5709276'), ('name', 'Helmerich Award')]
[('author', 'http://www.wikidata.org/entity/Q190379'), ('award', 'http://www.wikidata.org/e

9

Try to find the Nobel prize using regex on the entities belonging to literary award class.

In [11]:
#Q46
#i want to find the Nobel prize entity
queryString = """
SELECT DISTINCT ?award ?aname WHERE { 

?award wdt:P31 wd:Q378427.
#Labels
?award <http://schema.org/name> ?aname .
FILTER(CONTAINS(LCASE(?aname), "nobel")). 
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q37922'), ('aname', 'Nobel Prize in Literature')]


1

Retrieve some properties of Nobel Prize.

In [12]:
#Q47
#have a look at predicates and objects
queryString = """
SELECT DISTINCT ?p ?pname ?WHERE {

wd:Q37922 ?p ?o.

#Labels
?p <http://schema.org/name> ?pname .

} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('pname', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pname', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('pname', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P1705'), ('pname', 'native label')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pname', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('pname', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('pname', 'has list')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('pname', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pname', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/

33

Count how many authors have been nominated for Nobel Prize in literature.

In [13]:
#Q48
#authors nominated for Nobel Prize in literature
queryString = """
SELECT COUNT (?author) ?WHERE {

?author wdt:P1411 wd:Q37922.

} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '625')]


1

<h3>TASK 5 ENDS HERE</h3>
McEwan won 7 awards related to literature and got nominated 3 times, while Auster just won two. 
I also found the entity of Nobel Prize in Literature (wd:Q37922) and I discovered that both didn't got nominated for Nobel Prize in literature.